Definiamo il dataset Shoes di validation

In [1]:
import tensorflow as tf
import json
import os

# Definisci il percorso della cartella contenente le immagini
image_directory = 'C:/Users/marce/Desktop/polyvore_outfits/images/'

# Carica il file JSON delle etichette
json_file_path = 'Dataset/valid_set_bs.json'

with open(json_file_path, 'r') as json_file:
    label_data = json.load(json_file)

# Estrai le classi presenti nel file JSON
semantic_categories = set()
for entry in label_data:
    for item in entry['items']:
        semantic_category = item['semantic_category']
        semantic_categories.add(semantic_category)

# Crea una lista delle classi da considerare
class_names = ["tops", "bottoms", "shoes", "accessories"]
valid_labels = [label for label in class_names if label in semantic_categories]

# Crea un dizionario per mappare item_id alle etichette semantiche
item_id_to_label = {}
for entry in label_data:
    for item in entry['items']:
        item_id = item['item_id']
        semantic_category = item['semantic_category']
        if semantic_category in valid_labels:
            item_id_to_label[item_id] = semantic_category

# Crea una lista di tuple (nome dell'immagine, etichetta semantica)
image_label_pairs = []
for root, _, files in os.walk(image_directory):
    for file in files:
        # Estrai l'item_id dal nome dell'immagine
        item_id = os.path.splitext(file)[0]
        if item_id in item_id_to_label and item_id_to_label[item_id] == "shoes":
            image_path = os.path.join(root, file)
            image_label_pairs.append((image_path, "shoes"))

# Creare un dataset TensorFlow dalla lista di coppie immagine-etichetta
image_paths, labels = zip(*image_label_pairs)
image_paths = list(image_paths)
print(image_paths)
labels = list(labels)

# Converte le etichette semantiche in interi (necessario per poi poter effettuare i merge all'interno del generatore e del discriminatore)
label_mapping = {"shoes": 0}
def label_to_integer(label):
    return label_mapping[label]

labels = [label_to_integer(label) for label in labels]

# Creare un dataset TensorFlow
batch_size = 128
image_size = (300, 300)

def load_and_preprocess_image(image_path, label):
    # Carica e preelabora l'immagine
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, image_size)
    image = tf.cast(image, tf.uint8)
    return image, label

# Creare un dataset da coppie immagine-etichetta
dataset_val_shoes = tf.data.Dataset.from_tensor_slices((image_paths, labels))
dataset_val_shoes = dataset_val_shoes.map(load_and_preprocess_image)

# Per aggiungere il batch size alla dimensione del tensore per il dataset
dataset_val_shoes = dataset_val_shoes.batch(batch_size)

# Stampa il dataset
print(dataset_val_shoes)

['C:/Users/marce/Desktop/polyvore_outfits/images/100441415.jpg', 'C:/Users/marce/Desktop/polyvore_outfits/images/101272206.jpg', 'C:/Users/marce/Desktop/polyvore_outfits/images/101279079.jpg', 'C:/Users/marce/Desktop/polyvore_outfits/images/101288959.jpg', 'C:/Users/marce/Desktop/polyvore_outfits/images/101307780.jpg', 'C:/Users/marce/Desktop/polyvore_outfits/images/101309925.jpg', 'C:/Users/marce/Desktop/polyvore_outfits/images/101732849.jpg', 'C:/Users/marce/Desktop/polyvore_outfits/images/101837389.jpg', 'C:/Users/marce/Desktop/polyvore_outfits/images/102304500.jpg', 'C:/Users/marce/Desktop/polyvore_outfits/images/102519621.jpg', 'C:/Users/marce/Desktop/polyvore_outfits/images/102878938.jpg', 'C:/Users/marce/Desktop/polyvore_outfits/images/103022800.jpg', 'C:/Users/marce/Desktop/polyvore_outfits/images/103134574.jpg', 'C:/Users/marce/Desktop/polyvore_outfits/images/103178519.jpg', 'C:/Users/marce/Desktop/polyvore_outfits/images/103733228.jpg', 'C:/Users/marce/Desktop/polyvore_outfit

Richiamiamo l'embedding fisso nel progetto Shoes (funzione di creazione Embedding in notebook "Preprocess_%_Embedding"

In [2]:
import numpy as np

# Carica l'embedding dal file
embedding = np.load('embedding.npy')

# Ora puoi utilizzare l'embedding come desideri nel tuo notebook
print(embedding)

[[-0.6698174   0.36767587  2.0241685   0.59659916  0.6344788   0.02297916
   0.6140562   0.32278645  0.2504632  -0.6127753  -0.4668099   0.05412364
  -1.2268093  -0.23452778  0.31057438 -0.07198033  0.21593237 -0.55607826
   0.10858711  1.7606628  -0.7215787  -0.07988317 -1.3390868  -0.33724254
  -0.10062222  0.5519891   2.221586   -0.11137453  0.62013286 -0.7614147
   0.44494778 -0.07807393 -1.0250447   0.5582578  -0.80462873  0.57461697
   0.08267806 -1.3114728  -0.7518677   0.22460711  1.8143798   0.02875041
  -0.16374594  0.9635703  -0.9323846   0.63075733 -0.4927859   2.5482798
   0.25572893 -1.1241668  -0.27286592  0.8137388  -1.4529666   0.12695476
  -1.9686215   0.15296736 -1.482131    0.07811125  1.1749024  -0.54084545
  -0.58014643 -0.4272632  -0.85797626  0.11588039 -0.40775523  1.0385048
  -0.08945616  0.14564554  0.07501117 -0.48413223 -0.30717677 -0.19494504
   0.30911177 -0.57293016 -0.6120647   0.43311933 -0.5307508  -0.734938
  -0.8050627  -1.5378573  -0.47905245 -0.99

Creaimo il dataset di Training delle scarpe, la struttura è uguale al dataset di Validation 

In [2]:
import tensorflow as tf
import json
import os

# Definisci il percorso della cartella contenente le immagini
image_directory = 'C:/Users/marce/Desktop/polyvore_outfits/images/'

# Carica il file JSON delle etichette
json_file_path = 'Dataset/train_set_bs.json'

with open(json_file_path, 'r') as json_file:
    label_data = json.load(json_file)

# Estrai le classi presenti nel file JSON
semantic_categories = set()
for entry in label_data:
    for item in entry['items']:
        semantic_category = item['semantic_category']
        semantic_categories.add(semantic_category)

# Crea una lista delle classi da considerare
class_names = ["tops", "bottoms", "shoes", "accessories"]
valid_labels = [label for label in class_names if label in semantic_categories]

# Crea un dizionario per mappare item_id alle etichette semantiche
item_id_to_label = {}
for entry in label_data:
    for item in entry['items']:
        item_id = item['item_id']
        semantic_category = item['semantic_category']
        if semantic_category in valid_labels:
            item_id_to_label[item_id] = semantic_category

# Crea una lista di tuple (nome dell'immagine, etichetta semantica)
image_label_pairs = []
for root, _, files in os.walk(image_directory):
    for file in files:
        # Estrai l'item_id dal nome dell'immagine
        item_id = os.path.splitext(file)[0]
        if item_id in item_id_to_label and item_id_to_label[item_id] == "shoes":
            image_path = os.path.join(root, file)
            image_label_pairs.append((image_path, "shoes"))

# Creare un dataset TensorFlow dalla lista di coppie immagine-etichetta
image_paths, labels = zip(*image_label_pairs)
image_paths = list(image_paths)
print(image_paths)
labels = list(labels)

# Converte le etichette semantiche in interi (necessario per poi poter effettuare i merge all'interno del generatore e del discriminatore)
label_mapping = {"shoes": 0}
def label_to_integer(label):
    return label_mapping[label]

labels = [label_to_integer(label) for label in labels]

# Creare un dataset TensorFlow
batch_size = 128
image_size = (300, 300)

# La funzione di load_and_process_image è stata già definita precedentemente

# La funzione può essere non ridefinita se abbiamo già creato altri dataset in precedenza 

# Creare un dataset da coppie immagine-etichetta
dataset_shoes = tf.data.Dataset.from_tensor_slices((image_paths, labels))
dataset_shoes = dataset_shoes.map(load_and_preprocess_image)

# Per aggiungere il batch size alla dimensione del tensore per il dataset
dataset_shoes = dataset_shoes.batch(batch_size)

# Stampa il dataset
print(dataset_shoes)


['C:/Users/marce/Desktop/polyvore_outfits/images/100094985.jpg', 'C:/Users/marce/Desktop/polyvore_outfits/images/100116756.jpg', 'C:/Users/marce/Desktop/polyvore_outfits/images/100305898.jpg', 'C:/Users/marce/Desktop/polyvore_outfits/images/100338764.jpg', 'C:/Users/marce/Desktop/polyvore_outfits/images/100344935.jpg', 'C:/Users/marce/Desktop/polyvore_outfits/images/100357374.jpg', 'C:/Users/marce/Desktop/polyvore_outfits/images/100434551.jpg', 'C:/Users/marce/Desktop/polyvore_outfits/images/100459835.jpg', 'C:/Users/marce/Desktop/polyvore_outfits/images/100478779.jpg', 'C:/Users/marce/Desktop/polyvore_outfits/images/100500093.jpg', 'C:/Users/marce/Desktop/polyvore_outfits/images/100503275.jpg', 'C:/Users/marce/Desktop/polyvore_outfits/images/100630564.jpg', 'C:/Users/marce/Desktop/polyvore_outfits/images/100790382.jpg', 'C:/Users/marce/Desktop/polyvore_outfits/images/100790437.jpg', 'C:/Users/marce/Desktop/polyvore_outfits/images/100816505.jpg', 'C:/Users/marce/Desktop/polyvore_outfit

Faccio lo stesso lavoro per il test Set della categoria shoes

In [5]:
import tensorflow as tf
import json
import os

# Definisci il percorso della cartella contenente le immagini
image_directory = 'C:/Users/marce/Desktop/polyvore_outfits/images/'

# Carica il file JSON delle etichette
json_file_path = 'Dataset/test_set_bs.json'

with open(json_file_path, 'r') as json_file:
    label_data = json.load(json_file)

# Estrai le classi presenti nel file JSON
semantic_categories = set()
for entry in label_data:
    for item in entry['items']:
        semantic_category = item['semantic_category']
        semantic_categories.add(semantic_category)

# Crea una lista delle classi da considerare
class_names = ["tops", "bottoms", "shoes", "accessories"]
valid_labels = [label for label in class_names if label in semantic_categories]

# Crea un dizionario per mappare item_id alle etichette semantiche
item_id_to_label = {}
for entry in label_data:
    for item in entry['items']:
        item_id = item['item_id']
        semantic_category = item['semantic_category']
        if semantic_category in valid_labels:
            item_id_to_label[item_id] = semantic_category

# Crea una lista di tuple (nome dell'immagine, etichetta semantica)
image_label_pairs = []
for root, _, files in os.walk(image_directory):
    for file in files:
        # Estrai l'item_id dal nome dell'immagine
        item_id = os.path.splitext(file)[0]
        if item_id in item_id_to_label and item_id_to_label[item_id] == "shoes":
            image_path = os.path.join(root, file)
            image_label_pairs.append((image_path, "shoes"))

# Creare un dataset TensorFlow dalla lista di coppie immagine-etichetta
image_paths, labels = zip(*image_label_pairs)
image_paths = list(image_paths)
print(image_paths)
labels = list(labels)

# Converte le etichette semantiche in interi (necessario per poi poter effettuare i merge all'interno del generatore e del discriminatore)
label_mapping = {"shoes": 0}
def label_to_integer(label):
    return label_mapping[label]

labels = [label_to_integer(label) for label in labels]

# Creare un dataset TensorFlow
batch_size = 128
image_size = (300, 300)

#la funzione load_and_process_image è stata già definita precedentemente

# Creare un dataset da coppie immagine-etichetta
dataset_test_shoes = tf.data.Dataset.from_tensor_slices((image_paths, labels))
dataset_test_shoes = dataset_test_shoes.map(load_and_preprocess_image)

# Per aggiungere il batch size alla dimensione del tensore per il dataset
dataset_test_shoes = dataset_test_shoes.batch(batch_size)

# Stampa il dataset
print(dataset_test_shoes)


['C:/Users/marce/Desktop/polyvore_outfits/images/100010564.jpg', 'C:/Users/marce/Desktop/polyvore_outfits/images/100041924.jpg', 'C:/Users/marce/Desktop/polyvore_outfits/images/100068812.jpg', 'C:/Users/marce/Desktop/polyvore_outfits/images/100082887.jpg', 'C:/Users/marce/Desktop/polyvore_outfits/images/100086496.jpg', 'C:/Users/marce/Desktop/polyvore_outfits/images/100128872.jpg', 'C:/Users/marce/Desktop/polyvore_outfits/images/100134175.jpg', 'C:/Users/marce/Desktop/polyvore_outfits/images/100141912.jpg', 'C:/Users/marce/Desktop/polyvore_outfits/images/100177610.jpg', 'C:/Users/marce/Desktop/polyvore_outfits/images/100215754.jpg', 'C:/Users/marce/Desktop/polyvore_outfits/images/100218334.jpg', 'C:/Users/marce/Desktop/polyvore_outfits/images/100233102.jpg', 'C:/Users/marce/Desktop/polyvore_outfits/images/100244274.jpg', 'C:/Users/marce/Desktop/polyvore_outfits/images/100246962.jpg', 'C:/Users/marce/Desktop/polyvore_outfits/images/100262049.jpg', 'C:/Users/marce/Desktop/polyvore_outfit

Importo i vari package utili come Tensorflow, numpy, ecc..

In [6]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import preprocessing
from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import time
%matplotlib inline

from matplotlib import gridspec

Definisco la funzione di normalizzazione del dataset di training

In [7]:
import tensorflow as tf

def normalization(tensor):
    tensor = tf.image.resize(tensor, (128, 128))
    tensor = tf.subtract(tf.divide(tensor, 127.5), 1)
    return tensor

In [8]:
for image, label  in dataset_shoes.take(1):
    img = tf.cast(image, tf.float32)
    imgs = normalization(img)
    print(imgs.shape)
    print(label.shape)

(128, 128, 128, 3)
(128,)


Definisco la dimensione latente, hyperparameters importante per la CGAN

In [9]:
#BATCH_SIZE = 128
latent_dim = 128

In [10]:
# Etichetta da passare al generatore --> embedding maglietta
con_label = layers.Input(shape=128, name='input_embedding')
print(con_label)

# latent vector input
latent_vector = layers.Input(shape=latent_dim)
print(latent_vector)

KerasTensor(type_spec=TensorSpec(shape=(None, 128), dtype=tf.float32, name='input_embedding'), name='input_embedding', description="created by layer 'input_embedding'")
KerasTensor(type_spec=TensorSpec(shape=(None, 128), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'")


Normalizzo la forma del vettore di rumore e dell'etichetta per poterle concatenarle successivamente tra di loro senza problemi

In [11]:
from tensorflow.keras import layers
import numpy as np

def label_conditioned_generator():
    nodes = 128 * 4 * 4 
    label_dense = layers.Dense(nodes)(con_label)
    # reshape to additional channel
    label_reshape_layer = layers.Reshape((4, 4, 128))(label_dense)
    return label_reshape_layer

def latent_input(latent_dim=100):
    # image generator input
    nodes = 512 * 4 * 4
    latent_dense = layers.Dense(nodes)(latent_vector)
    latent_dense = layers.ReLU()(latent_dense)
    latent_reshape = layers.Reshape((4, 4, 512))(latent_dense)
    return latent_reshape


In [12]:
# define the final generator model
def define_generator():
    label_output = label_conditioned_generator()
    #label_output = layers.Input(shape=(embedding_resnet.shape))
    latent_vector_output= latent_input()
    # merge label_conditioned_generator and latent_input output
    merge = layers.Concatenate()([latent_vector_output, label_output])

    x = layers.Conv2DTranspose(64 * 8, kernel_size=4, strides= 2, padding='same', kernel_initializer=tf.keras.initializers.RandomNormal(
    mean=0.0, stddev=0.02), use_bias=False, name='conv_transpose_1')(merge)
    x = layers.BatchNormalization(momentum=0.1,  epsilon=0.8, center=1.0, scale=0.02, name='bn_1')(x)
    x = layers.ReLU(name='relu_1')(x)

    x = layers.Conv2DTranspose(64 * 4, kernel_size=4, strides= 2, padding='same', kernel_initializer=tf.keras.initializers.RandomNormal(
    mean=0.0, stddev=0.02), use_bias=False, name='conv_transpose_2')(x)
    x = layers.BatchNormalization(momentum=0.1,  epsilon=0.8, center=1.0, scale=0.02, name='bn_2')(x)
    x = layers.ReLU(name='relu_2')(x)

    x = layers.Conv2DTranspose(64 * 2, 4, 2, padding='same', kernel_initializer=tf.keras.initializers.RandomNormal(
    mean=0.0, stddev=0.02), use_bias=False, name='conv_transpose_3')(x)
    x = layers.BatchNormalization(momentum=0.1,  epsilon=0.8,  center=1.0, scale=0.02, name='bn_3')(x)
    x = layers.ReLU(name='relu_3')(x)


    x = layers.Conv2DTranspose(64 * 1, 4, 2, padding='same', kernel_initializer=tf.keras.initializers.RandomNormal(
    mean=0.0, stddev=0.02), use_bias=False, name='conv_transpose_4')(x)
    x = layers.BatchNormalization(momentum=0.1,  epsilon=0.8,  center=1.0, scale=0.02, name='bn_4')(x)
    x = layers.ReLU(name='relu_4')(x)


    out_layer = layers.Conv2DTranspose(3, 4, 2,padding='same', kernel_initializer=tf.keras.initializers.RandomNormal(
    mean=0.0, stddev=0.02), use_bias=False, activation='tanh', name='conv_transpose_6')(x)

   # define model
    model = tf.keras.Model([latent_vector, con_label], out_layer)
    return model

conditional_gen = define_generator()
conditional_gen.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128)]                0         []                            
                                                                                                  
 dense_1 (Dense)             (None, 8192)                 1056768   ['input_1[0][0]']             
                                                                                                  
 input_embedding (InputLaye  [(None, 128)]                0         []                            
 r)                                                                                               
                                                                                                  
 re_lu (ReLU)                (None, 8192)                 0         ['dense_1[0][0]']         

L'input del condizionatore sono l'etichetta del dataset, e l'immagine creata dal generatore

In [13]:
def label_condition_disc(in_shape=(128,128,3), n_classes=1, embedding_dim=128):
    # label input (sarebbe la mia etichetta, cioè bottom,shoes o accesories, in questo caso solo shoes)
    etichetta = layers.Input(shape=(1,), )
    label_embedding = layers.Embedding(n_classes, embedding_dim)(etichetta)
    # scale up to image dimensions with linear activation
    nodes = in_shape[0] * in_shape[1] * in_shape[2]
    label_dense = layers.Dense(nodes)(label_embedding)
    # reshape to additional channel
    label_reshape_layer = layers.Reshape((in_shape[0], in_shape[1], 3))(label_dense)
    # image input
    return etichetta, label_reshape_layer

# image_disc è l'immagine generata che passo al discriminatore
def image_disc(in_shape=(128,128,3)):
    inp_image = layers.Input(shape=in_shape)
    return inp_image

Definiamo la funzione del discriminatore

In [14]:
def define_discriminator():
    etichetta, label_condition_output = label_condition_disc()
    inp_image_output = image_disc()
    # concat label as a channel
    merge = layers.Concatenate(axis=-1)([inp_image_output, label_condition_output])

    x = layers.Conv2D(64, kernel_size=4, strides= 2, padding='same', kernel_initializer=tf.keras.initializers.RandomNormal(
    mean=0.0, stddev=0.02), use_bias=False, name='conv_1')(merge)
    x = layers.LeakyReLU(0.2, name='leaky_relu_1')(x)

    x = layers.Conv2D(64 * 2, kernel_size=4, strides= 3, padding='same', kernel_initializer=tf.keras.initializers.RandomNormal(
    mean=0.0, stddev=0.02), use_bias=False, name='conv_2')(x)
    x = layers.BatchNormalization(momentum=0.1,  epsilon=0.8, center=1.0, scale=0.02, name='bn_1')(x)
    x = layers.LeakyReLU(0.2, name='leaky_relu_2')(x)

    x = layers.Conv2D(64 * 4, 4, 3, padding='same', kernel_initializer=tf.keras.initializers.RandomNormal(
    mean=0.0, stddev=0.02), use_bias=False, name='conv_3')(x)
    x = layers.BatchNormalization(momentum=0.1,  epsilon=0.8, center=1.0, scale=0.02, name='bn_2')(x)
    x = layers.LeakyReLU(0.2, name='leaky_relu_3')(x)

    x = layers.Conv2D(64 * 8, 4, 3,padding='same', kernel_initializer=tf.keras.initializers.RandomNormal(
    mean=0.0, stddev=0.02), use_bias=False, name='conv_5')(x)
    x = layers.BatchNormalization(momentum=0.1,  epsilon=0.8, center=1.0, scale=0.02, name='bn_4')(x)
    x = layers.LeakyReLU(0.2, name='leaky_relu_5')(x)


    flattened_out = layers.Flatten()(x)
    # dropout
    dropout = layers.Dropout(0.4)(flattened_out)
    # output
    dense_out = layers.Dense(1, activation='sigmoid')(dropout)
    # define model


    # define model
    model = tf.keras.Model([inp_image_output, etichetta], dense_out)
    return model

# Inizializziamo il discriminatore CGAN
conditional_discriminator = define_discriminator()

# Sommario discriminatore CGAN
conditional_discriminator.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 embedding (Embedding)       (None, 1, 128)               128       ['input_2[0][0]']             
                                                                                                  
 dense_2 (Dense)             (None, 1, 49152)             6340608   ['embedding[0][0]']           
                                                                                                  
 input_3 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                            

Come loss scelgo la binary croos entropy, molto utilizzata nelle casistiche di classificazione della CGAN

In [15]:
binary_cross_entropy = tf.keras.losses.BinaryCrossentropy()

Definisco la loss del generatore e del discriminatore

In [16]:
def generator_loss(label, fake_output):
    gen_loss = binary_cross_entropy(label, fake_output)
    print('gen loss --> ', gen_loss)
    return gen_loss

In [17]:
def discriminator_loss(label, output):
    disc_loss = binary_cross_entropy(label, output)
    print('disc loss --> ', disc_loss)
    return disc_loss

Impostiamo il learning rate e gli ottimizzatori del generatore e del discriminatore, utilizzeremo ADAM

In [18]:
larning_rate = 0.0002
generator_optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0002, beta_1 = 0.5, beta_2 = 0.999 )
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0002, beta_1 = 0.5, beta_2 = 0.999 )

# Per il generatore
generator_variables = conditional_gen.trainable_variables

# Per il discriminatore
discriminator_variables = conditional_discriminator.trainable_variables


In [19]:
num_examples_to_generate = 1
# Definiamo il seme di generazione del nostro rumore
seed = tf.random.normal([num_examples_to_generate, latent_dim])

In [27]:
# Inizializziamo le variabili per tenere traccia delle somme cumulative
total_gen_loss = 0.0
total_disc_loss2 = 0.0

Definisco la funzione di train_step che lavora sull'apprendimento della CGAN tramite gradient descent

In [62]:
# Il target è l'etichetta reale dell'immagine
def train_step(images,target, G_loss_list, D_loss_list):
    
    # noise vector sampled from normal distribution
    # Crea il vettore di 3 dimensioni con valori casuali da una distribuzione normale
    #print('target:',target.shape[0])
    noise = tf.random.normal([target.shape[0], latent_dim])
    # Train Discriminator with real labels
    with tf.GradientTape() as disc_tape1:
        # Ottieni la dimensione effettiva del vettore target
        dimensione_target = tf.shape(target)[0]
        # Modifico la dimensione del batch size dell'embedding in base alla dimensione attuale del batch_size
        new_embedding = tf.tile(embedding, [dimensione_target, 1])
        # Genero l'immagine fake col gen passando il vettore di rumore e l'embedding dell'immagine
        generated_image = conditional_gen([noise,new_embedding], training=True)

        real_output = conditional_discriminator([images,target], training=True)
        real_targets = tf.ones_like(real_output)
        disc_loss1 = discriminator_loss(real_targets, real_output)

    # Calcolo del gradiente per il discriminatore per l'etichette reali 
    gradients_of_disc1 = disc_tape1.gradient(disc_loss1, conditional_discriminator.trainable_variables)

    # Ottimizzazione dei parametri del discriminatore per l'etichette reali 
    discriminator_optimizer.apply_gradients(zip(gradients_of_disc1,\
    conditional_discriminator.trainable_variables))

    # Training del discriminatore con l'etichette false
    with tf.GradientTape() as disc_tape2:
        #print('fake output discriminator')
        fake_output = conditional_discriminator([generated_image,target], training=True)
        fake_targets = tf.zeros_like(fake_output)
        disc_loss2 = discriminator_loss(fake_targets, fake_output)
        
    # Calcolo del gradiente per il discriminatore per l'etichette reali
    gradients_of_disc2 = disc_tape2.gradient(disc_loss2, conditional_discriminator.trainable_variables)

    # Ottimizzazione dei parametri del discriminatore per l'etichette false
    discriminator_optimizer.apply_gradients(zip(gradients_of_disc2,\
    conditional_discriminator.trainable_variables))

    # Fase di Train del Generatore con l'etichette reali
    with tf.GradientTape() as gen_tape:
        generated_image = conditional_gen([noise,new_embedding], training=True)
        fake_output = conditional_discriminator([generated_image,target], training=True)
        real_targets = tf.ones_like(fake_output)
        gen_loss = generator_loss(real_targets, fake_output)

    # Calcolo del gradiente per il generatore per l'etichette reali
    gradients_of_gen = gen_tape.gradient(gen_loss, conditional_gen.trainable_variables)

    # Ottimizzazione dei parametri del generatore per l'etichette reali
    generator_optimizer.apply_gradients(zip(gradients_of_gen,\
    conditional_gen.trainable_variables))
    
    G_loss_list.append(gen_loss)
    D_loss_list.append(disc_loss2)
    
    # Accumiamo le losses per ogni step
    global total_gen_loss
    global total_disc_loss2
    total_gen_loss += gen_loss
    total_disc_loss2 += disc_loss2

Definiamo le funzione per il calcolo del FID e Inception Score

In [24]:
from keras.src.utils import img_to_array, load_img
!pip install tensorflow-probability

# Importa librerie e modelli necessari
import tensorflow as tf
import tensorflow_probability as tfp
from sklearn.metrics import mutual_info_score
from scipy.linalg import sqrtm
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
import numpy as np

from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model

# Importazioni delle librerie e dei modelli necessari
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model

# Funzione per estrarre le features da un set di immagini utilizzando il modello InceptionV3
    
def inception_features(images):
    # Carica il modello InceptionV3 preaddestrato
    base_model = InceptionV3(weights='imagenet', input_shape=(299, 299, 3))
    model = Model(inputs=base_model.input, outputs=base_model.get_layer('avg_pool').output)

    # Ingrandisci e pre-elabora le immagini
    img = tf.image.resize(images, (299, 299))
    preprocessed_images = preprocess_input(img)

    # Utilizza il modello InceptionV3 per estrarre le features dalle immagini
    features = model.predict(preprocessed_images)

    return features


# Funzione per calcolare il FID
# Funzione per calcolare il FID tra due insiemi di immagini
def calculate_fid(real_images, generated_images):
    # Calcola le statistiche delle feature dalle immagini reali e generate
    real_features = inception_features(real_images)
    generated_features = inception_features(generated_images)

    # Calcola la media e la covarianza delle feature delle immagini reali e generate
    mu_real, sigma_real = tf.math.reduce_mean(real_features, axis=0), tfp.stats.covariance(real_features)
    mu_generated, sigma_generated = tf.math.reduce_mean(generated_features, axis=0), tfp.stats.covariance(generated_features)

    # Calcola la traccia della radice quadrata delle matrici di covarianza
    sqrt_term = sqrtm(sigma_real @ sigma_generated)

    # Calcola la differenza tra le medie delle feature
    diff = mu_real - mu_generated

    # Calcola la distanza di Fréchet
    # Ho modificato la normalizzazione da "frobenius" che è una norma non supportata da tensorflow in "euclidean"
    fid = tf.math.real(tf.linalg.trace(sigma_real + sigma_generated - 2.0 * sqrt_term)) + tf.norm(diff, ord='euclidean')**2
    return fid


# Funzione per calcolare l'Inception Score
def calculate_inception_score(generated_images):
    # Carica il modello InceptionV3 preaddestrato senza gli strati finali
    inception_model = InceptionV3(include_top=False, weights='imagenet')

    # Estrai le feature dalle immagini generate
    features = inception_model.predict(generated_images)

    # Calcola le previsioni di classe per ogni immagine
    class_predictions = tf.nn.softmax(features, axis=-1)

    # Calcola l'entropia delle previsioni di classe
    entropy = -tf.reduce_sum(class_predictions * tf.math.log(class_predictions + 1e-10), axis=-1)

    # Calcola l'Inception Score come media delle entropie delle previsioni di classe
    inception_score = tf.math.exp(tf.reduce_mean(entropy))
    return inception_score

You should consider upgrading via the 'C:\Users\marce\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [25]:
# Funzione per valutare il modello utilizzando FID e Inception Score
def evaluate_model(conditional_gen, validation_dataset):
    fid_scores = []
    inception_scores = []

    # Itera attraverso il set di dati di validazione
    for batch_images, batch_targets in validation_dataset:
        img = tf.cast(batch_images, tf.float32)
        imgs = normalization(img)
        
        # Genera le immagini condizionalmente
        noise = tf.random.normal([batch_targets.shape[0], latent_dim])
        dimensione_target = tf.shape(batch_targets)[0]
        new_embedding = tf.tile(embedding, [dimensione_target, 1])
        generated_images = conditional_gen([noise, new_embedding], training=False)

        # Calcola il FID tra le immagini reali e generate utilizzando il validation_dataset
        print('imgs shape', imgs.shape)
        fid = calculate_fid(imgs, generated_images)
        fid_scores.append(fid)

        # Calcola l'Inception Score per le immagini generate
        inception_score = calculate_inception_score(generated_images)
        inception_scores.append(inception_score)

    # Calcola la media dei punteggi FID e Inception Score
    average_fid = np.mean(fid_scores)
    average_inception_score = np.mean(inception_scores)

    return average_fid, average_inception_score

Definiamo la funzione di train che itera sull'intero dataset impostando le epoche. All'interno, ogni tot epoche verrà effettuata la valutazione del modello tramite FID e INCEPTION score

In [65]:
def train(dataset, epochs, validation_dataset):
    
    # Verifica se la directory per il salvataggio dei pesi esiste, altrimenti creala
    save_dir = 'Dataset/training_weights/Shoes/Params_Tuning/batch_size'
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    #Definisci la frequenza di valutazione sul set di validazione (ad esempio, ogni 10 epoche)
    validation_interval = 10
    for epoch in range(epochs):
        start = time.time()
        i = 0
        D_loss_list, G_loss_list = [], []
        fid_list, inception_list = [], []
        # Ciclo for che attraversa il mio dataset con il dataset che è una collezione di coppie di dati
        for image, label in dataset:
            # Non devo filtrare niente perchè il dataset è formato da sole immagini di shoes
                i += 1
                img = tf.cast(image, tf.float32)
                imgs = normalization(img)
                train_step(imgs, label, G_loss_list, D_loss_list)
        # Valuta il modello sul set di validazione alla fine di ogni epoca
        if (epoch + 1) % validation_interval == 0:
            average_fid, average_inception_score = evaluate_model(conditional_gen, validation_dataset)
            print(f'Epoch {epoch + 1} - Average FID: {average_fid}, Average Inception Score: {average_inception_score}')
            fid_list.append(average_fid)
            inception_list.append(average_inception_score)
        #display.clear_output(wait=True)
        generate_and_save_images(conditional_gen, epoch + 1, seed, save_dir='/content/train/')

        # Salva i pesi del modello ogni 10 epoche
        if (epoch + 1) % 10 == 0:
            conditional_gen.save_weights('Dataset/training_weights/Shoes/Params_Tuning/batch_size/gen_'+ str(epoch)+'.h5')
            conditional_discriminator.save_weights('Dataset/training_weights/Shoes/Params_Tuning/batch_size/disc_'+ str(epoch)+'.h5')

        print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

    # Genera dopo l'ultima epoca
    #display.clear_output(wait=True)
    generate_and_save_images(conditional_gen,
                            epochs,
                            seed, save_dir='/content/train/')
    
    print('Fid List : ', fid_list)
    print('Inception list: ', inception_list)
    
    avg_gen_loss = total_gen_loss / epochs  
    avg_disc_loss2 = total_disc_loss2 / epochs  

    print("Average Generator Loss:", avg_gen_loss)
    print("Average Discriminator Loss2:", avg_disc_loss2)
    
    return D_loss_list, G_loss_list


Definisco la funzione per generare graficamente l'immagine

In [66]:
# Test input --> seed

import os

def generate_and_save_images(model, epoch, test_input, save_dir):
    # Creare la directory se non esiste
    os.makedirs(save_dir, exist_ok=True)

    # nota che `training` è settato su False.
    # Questo perchè tutti i layers vengono eseguiti in inference mode (batchnorm).

    # Io al generatore non passo l'etichetta tops,bottoms,shoes,ecc ma gli passo il mio embedding

    predictions = model([test_input, embedding], training=False)
    print(predictions.shape)
    fig = plt.figure(figsize=(25, 25))

    print("Generated Images are Conditioned on Embedding of Tops:")
    for i in range(predictions.shape[0]):
        plt.subplot(5, 5, i + 1)
        pred = (predictions[i, :, :, :] + 1) * 127.5
        pred = np.array(pred)
        plt.imshow(pred.astype(np.uint8))
        plt.axis('off')

    # Salvare l'immagine nella directory specificata
    image_path = os.path.join(save_dir, 'image_at_epoch_{:d}.png'.format(epoch))
    plt.savefig(image_path)
    plt.show()



In [ ]:
train(dataset_shoes,30, dataset_val_shoes) 

Carichiamo i pesi del training fatto in precedenza per continuare il training in un secondo momento

In [26]:
conditional_gen.load_weights('Dataset/training_weights/Shoes/gen_14.h5')
conditional_discriminator.load_weights('Dataset/training_weights/Shoes/disc_14.h5')

Richiamiamo di nuovo la funzione training per altre 30 epoche

In [ ]:
train(dataset_shoes,30, dataset_val_shoes) 

Ricarico i pesi della 35° epoca nel generatore e discriminatore per continuare il training

In [23]:
conditional_gen.load_weights('Dataset/training_weights/Shoes/gen_19.h5')
conditional_discriminator.load_weights('Dataset/training_weights/Shoes/disc_19.h5')

In [ ]:
train(dataset_shoes, 20, dataset_val_shoes)

Funzione per generare e visualizzare l'immagine finale risultante dal training

In [62]:
def display_image(image):
    # Visualizza un'immagine senza griglia.
    plt.imshow(image)
    plt.axis('off')
    plt.show()

def generate_and_display_image(model, test_input):
    # Imposta training su False per l'inferenza.
    output = model([test_input, embedding], training=False)

    # Normalizza e converte l'output in un array numpy di immagine.
    pred = (output[0, :, :, :] + 1 ) * 127.5
    pred = np.array(pred, dtype=np.uint8)

    # Visualizza l'immagine risultato.
    display_image(pred)


In [63]:
num_examples_to_generate = 1
latent_dim = 100
noise = tf.random.normal([num_examples_to_generate, latent_dim])

In [ ]:
generate_images(conditional_gen, noise)

Training con latent_dim = 128 e learning_rate = 0.0002

In [ ]:
train(dataset_shoes, 30, dataset_val_shoes)

In [35]:
conditional_gen.load_weights('Dataset/training_weights/Shoes/Params_Tuning/latent_dim/gen_19.h5')
conditional_discriminator.load_weights('Dataset/training_weights/Shoes/Params_Tuning/latent_dim/disc_19.h5')

In [ ]:
train(dataset_shoes, 10, dataset_val_shoes)

Ora lavoro con batch_size=64 e non 128, lr=0.0002 e latent_dim = 100

In [ ]:
train(dataset_shoes, 30, dataset_val_shoes)

Carichiamo i pesi del generatore e discriminatore con i parametri sintonizzati (latent_dim = 128)

In [21]:
conditional_gen.load_weights('Dataset/training_weights/Shoes/Params_Tuning/latent_dim/gen_19.h5')
conditional_discriminator.load_weights('Dataset/training_weights/Shoes/Params_Tuning/latent_dim/disc_19.h5')

Valutiamo le metriche di performance sul test set 

In [26]:
average_fid, average_inception_score = evaluate_model(conditional_gen, dataset_test_shoes)
print(f' - Average FID: {average_fid}, Average Inception Score: {average_inception_score}')

imgs shape (128, 128, 128, 3)
4/4 [==============================] - 7s 1s/step


C:\Users\marce\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow\python\ops\gen_math_ops.py:12924: ComplexWarning: Casting complex values to real discards the imaginary part
  _result = pywrap_tfe.TFE_Py_FastPathExecute(


4/4 [==============================] - 2s 236ms/step
imgs shape (128, 128, 128, 3)
4/4 [==============================] - 2s 234ms/step
imgs shape (128, 128, 128, 3)
4/4 [==============================] - 3s 242ms/step
imgs shape (128, 128, 128, 3)
4/4 [==============================] - 2s 237ms/step
imgs shape (128, 128, 128, 3)
4/4 [==============================] - 3s 240ms/step
imgs shape (128, 128, 128, 3)
4/4 [==============================] - 2s 242ms/step
imgs shape (128, 128, 128, 3)
4/4 [==============================] - 2s 238ms/step
imgs shape (128, 128, 128, 3)
4/4 [==============================] - 3s 247ms/step
imgs shape (128, 128, 128, 3)
4/4 [==============================] - 2s 232ms/step
imgs shape (128, 128, 128, 3)
4/4 [==============================] - 2s 242ms/step
imgs shape (128, 128, 128, 3)
4/4 [==============================] - 2s 234ms/step
imgs shape (128, 128, 128, 3)
4/4 [==============================] - 2s 239ms/step
imgs shape (128, 128, 128, 3)
4/4 